In [2]:
import pandas as pd
import time

from sqlalchemy import create_engine, text

In [3]:
dict_conn = {'username': 'root', 
                 'password': '1234',
                 'host': 'localhost',
                 'port': '3306',
                 'database': 'discogs'}

In [4]:
# Создаем строку подключения (используя, например, pymysql)
connection_string = f"mysql+pymysql://{dict_conn['username']}:{dict_conn['password']}@{dict_conn['host']}:{dict_conn['port']}/{dict_conn['database']}"
engine = create_engine(connection_string)

In [5]:
def get_df(select_code):
    try:
        start_time = time.time()
        df = pd.read_sql(select_code, con=engine)
        end_time = time.time()
        
        execution_time = end_time - start_time
        print(f"Время выполнения запроса: {execution_time:.4f} секунд")
    except Exception as e:
        print(f"Ошибка: {e}")
        
    return df

In [6]:
def get_query_db(query):
    try:
        with engine.connect() as connection:
            start_time = time.time()
            result = connection.execute(query)
            end_time = time.time()
            
            execution_time = end_time - start_time
            print(f"Время выполнения запроса: {execution_time:.4f} секунд")
    except Exception as e:
        print(f"Ошибка: {e}")
    

In [7]:
def get_index(table):
    query_index = text(f"""show indexes from {table};""")  
    df_index = get_df(query_index)
    return df_index

In [8]:
def drop_index(index_name, table):
    query_drop = text(f"""drop INDEX {index_name} ON {table}""")
    get_query_db(query_drop)

----------------------------------

In [11]:
get_index('artist')

Время выполнения запроса: 0.0071 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,artist,0,PRIMARY,1,ARTIST_ID,A,6291116,None,None,,BTREE,,,YES,None
1,artist,1,idx_artist_id,1,ARTIST_ID,A,6291116,None,None,,BTREE,,,YES,None
2,artist,1,ft_profile,1,PROFILE,None,6291116,None,None,YES,FULLTEXT,,,YES,None


In [12]:
# Найти информацию по заданному исполнителю, используя его имя.

query_1 = text("""select * from artist 
                where name = 'K.A.B.'""")  

get_df(query_1)

Время выполнения запроса: 21.1554 секунд


,ARTIST_ID,NAME,REALNAME,PROFILE,DATA_QUALITY
0,6,K.A.B.,Karl Axel Bissler,,Needs Major Changes


In [13]:
query = text("""CREATE INDEX idx_artist_name ON artist (name);""")
df = get_query_db(query)

Время выполнения запроса: 68.7217 секунд


In [14]:
# после создания индекса
get_df(query_1)

Время выполнения запроса: 0.0080 секунд


,ARTIST_ID,NAME,REALNAME,PROFILE,DATA_QUALITY
0,6,K.A.B.,Karl Axel Bissler,,Needs Major Changes


In [10]:
drop_index('idx_artist_name', 'artist')

Время выполнения запроса: 0.0894 секунд


-------------------


In [111]:
get_index('`group`')

Время выполнения запроса: 0.0038 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,group,0,PRIMARY,1,GROUP_ID,A,1285836,None,None,,BTREE,,,YES,None


In [110]:
get_index('`artist`')

Время выполнения запроса: 0.0665 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,artist,0,PRIMARY,1,ARTIST_ID,A,6291116,None,None,,BTREE,,,YES,None
1,artist,1,idx_artist_id,1,ARTIST_ID,A,6291116,None,None,,BTREE,,,YES,None
2,artist,1,ft_profile,1,PROFILE,None,6291116,None,None,YES,FULLTEXT,,,YES,None


In [104]:
# Найти всех участников указанного музыкального коллектива (по названию коллектива).

query_2 = text("""SELECT * 
                FROM artist 
                JOIN `group` ON artist.artist_id = `group`.group_artist_id
                WHERE `group`.main_artist_id = (SELECT artist_id FROM artist WHERE name = 'DATacide');""")  

get_df(query_2)

Время выполнения запроса: 14.1557 секунд


,ARTIST_ID,NAME,REALNAME,PROFILE,DATA_QUALITY,GROUP_ID,MAIN_ARTIST_ID,GROUP_ARTIST_ID
0,25,Tetsu Inoue,Tetsu Inoue,,Needs Vote,47,22,25
1,519207,Uwe Schmidt,Uwe H. Schmidt,"Born August 27th, 1968 in Frankfurt am Main, G...",Correct,263452,22,519207


In [ ]:
query_index_1 = text("""CREATE INDEX idx_group_main_artist_id ON discogs.group (main_artist_id);""")
query_index_2 = text("""CREATE INDEX idx_group_artist_id ON discogs.group (group_artist_id);""")

get_query_db(query_index_1)
get_query_db(query_index_2)

Время выполнения запроса: 4.6425 секунд
Время выполнения запроса: 4.6870 секунд


In [113]:
get_df(query_2)

Время выполнения запроса: 11.4961 секунд


,ARTIST_ID,NAME,REALNAME,PROFILE,DATA_QUALITY,GROUP_ID,MAIN_ARTIST_ID,GROUP_ARTIST_ID
0,25,Tetsu Inoue,Tetsu Inoue,,Needs Vote,47,22,25
1,519207,Uwe Schmidt,Uwe H. Schmidt,"Born August 27th, 1968 in Frankfurt am Main, G...",Correct,263452,22,519207


In [114]:
drop_index('idx_group_main_artist_id', '`group`')
drop_index('idx_group_artist_id', '`group`')

Время выполнения запроса: 0.0899 секунд
Время выполнения запроса: 0.0209 секунд


--------------


In [115]:
get_index('`release`')

Время выполнения запроса: 0.0051 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,release,0,PRIMARY,1,RELEASE_ID,A,10657532,None,None,,BTREE,,,YES,None
1,release,1,idx_release_released,1,RELEASED,A,24948,None,None,,BTREE,,,YES,None
2,release,1,idx_release_main_release,1,IS_MAIN_RELEASE,A,1,None,None,,BTREE,,,YES,None
3,release,1,idx_release_year,1,RELEASED,A,29808,None,None,,BTREE,,,YES,None


In [116]:
get_index('`release_artist`')

Время выполнения запроса: 0.0041 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,release_artist,0,PRIMARY,1,RELEASE_ARTIST_ID,A,52563772,None,None,,BTREE,,,YES,None
1,release_artist,1,idx_release_artist_id,1,ARTIST_ID,A,3519676,None,None,,BTREE,,,YES,None


In [19]:
# Индексы 3
# Найти все релизы заданного исполнителя и отсортировать их по дате выпуска. 
# Вывести имя исполнителя, название релиза, дату выхода.

query_indx_3 = text("""explain SELECT artist.name, r.title, r.released 
                        FROM discogs.release r
                        JOIN release_artist ON r.release_id = release_artist.release_id
                        JOIN artist ON artist.artist_id = release_artist.artist_id
                        WHERE artist.name = 'K.A.B.'
                        ORDER BY r.released;""")  

get_df(query_indx_3)

Время выполнения запроса: 0.0672 секунд


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,artist,None,ref,"PRIMARY,idx_artist_id,idx_artist_name",idx_artist_name,1022,const,1,100.0,Using index; Using temporary; Using filesort
1,1,SIMPLE,release_artist,None,ALL,idx_release_id,None,None,None,52563772,10.0,Using where; Using join buffer (hash join)
2,1,SIMPLE,r,None,eq_ref,PRIMARY,PRIMARY,4,discogs.release_artist.RELEASE_ID,1,100.0,None


In [18]:
query_2 = text("""CREATE INDEX idx_release_id ON discogs.release_artist (release_id);""")

get_query_db(query_2)

Время выполнения запроса: 145.5880 секунд


In [ ]:
query_1 = text("""CREATE INDEX idx_release_artist_id ON discogs.release_artist (artist_id);""")
query_2 = text("""CREATE INDEX idx_release_released ON discogs.release (released);""")

get_query_db(query_1)
get_query_db(query_2)

Время выполнения запроса: 151.6907 секунд
Время выполнения запроса: 38.3926 секунд


In [ ]:
query_2 = text("""explain ;""")

get_query_db(query_1)

In [ ]:
get_df(query_indx_3)

Время выполнения запроса: 0.0410 секунд


In [ ]:
drop_index('idx_release_artist_id', '`release_artist`')
drop_index('idx_release_released', '`release`')

Время выполнения запроса: 0.1147 секунд
Время выполнения запроса: 0.0301 секунд


---------------------

In [62]:
get_index('`release`')

Время выполнения запроса: 0.0083 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,release,0,PRIMARY,1,RELEASE_ID,A,10657532,None,None,,BTREE,,,YES,None
1,release,1,idx_release_released,1,RELEASED,A,24948,None,None,,BTREE,,,YES,None


In [63]:
get_index('`style`')

Время выполнения запроса: 0.0067 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,style,0,PRIMARY,1,STYLE_ID,A,17868512,None,None,,BTREE,,,YES,None


In [ ]:
# Индексы 4
# Найти все главные релизы, выпущенные в указанный год, с указанием стиля релиза. 
# Релиз является главным, если поле release.IS_MAIN_RELEASE = 1.

query_indx_4 = text("""explain SELECT r.title, style.style_name
                        FROM discogs.release r
                        JOIN style ON r.release_id = style.release_id
                        WHERE r.is_main_release = 1 
                        AND YEAR(r.released) = 2023
                        AND STYLE_NAME = 'Deep House';""")  

get_df(query_indx_4)

Время выполнения запроса: 0.0020 секунд


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,style,None,ALL,None,None,None,None,17868513,10.0,Using where
1,1,SIMPLE,r,None,eq_ref,PRIMARY,PRIMARY,4,discogs.style.RELEASE_ID,1,10.0,Using where


In [65]:
query_1 = text("""CREATE INDEX idx_release_main_release ON discogs.release (is_main_release);""")
query_2 = text("""CREATE INDEX idx_release_year ON discogs.release (released);""")
query_3 = text("""CREATE INDEX idx_release_style ON discogs.style (release_id);""")

get_query_db(query_1)
get_query_db(query_2)
get_query_db(query_3)

Время выполнения запроса: 37.0823 секунд
Время выполнения запроса: 37.0634 секунд
Время выполнения запроса: 49.5503 секунд


In [66]:
get_query_db(query_indx_4)

Время выполнения запроса: 32.0790 секунд


In [61]:
drop_index('idx_release_main_release', '`release`')
drop_index('idx_release_year', '`release`')
drop_index('idx_release_style', '`style`')

Время выполнения запроса: 0.0990 секунд
Время выполнения запроса: 0.0445 секунд
Время выполнения запроса: 0.0333 секунд


------------------

In [86]:
get_index('artist')

Время выполнения запроса: 0.0776 секунд


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,artist,0,PRIMARY,1,ARTIST_ID,A,6291116,None,None,,BTREE,,,YES,None
1,artist,1,idx_artist_name,1,NAME,A,6291116,None,None,,BTREE,,,YES,None
2,artist,1,ft_profile,1,PROFILE,None,6291116,None,None,YES,FULLTEXT,,,YES,None


In [83]:
# Индексы 5
# Найти всех исполнителей, в описании (профиле) которых встречается указанное выражение, 
# с использованием полнотекстового запроса.

query_indx_5= text("""
SELECT *
FROM discogs.artist
WHERE PROFILE LIKE '%рок%';""") 
get_query_db(query_indx_5)

Время выполнения запроса: 23.3056 секунд


In [85]:
index_5 = """ALTER TABLE discogs.artist ADD FULLTEXT INDEX ft_profile (PROFILE);"""

get_query_db(index_5)

Ошибка: Not an executable object: 'ALTER TABLE discogs.artist ADD FULLTEXT INDEX ft_profile (PROFILE);'


In [87]:
query_indx_5 = """SELECT *
    FROM discogs.artist
    WHERE MATCH(PROFILE) AGAINST('рок' IN NATURAL LANGUAGE MODE);"""

get_df(query_indx_5)

Время выполнения запроса: 0.2727 секунд


,ARTIST_ID,NAME,REALNAME,PROFILE,DATA_QUALITY
0,4239166,Крекерс,Krekers,Группа Крекерс (Krekers) из Санкт-Петербурга —...,Needs Vote
1,5107995,Гоген,,"Группа ""Гоген"" образовалась весной 1988 года, ...",Needs Vote
2,472264,Тамбурин,,"Группу создал певец, композитор, аранжировщик,...",Needs Vote
3,1620685,Гласовете,,"Групата „ГЛАСОВЕТЕ”, (или както е популярно то...",Needs Vote
4,2350272,Анатолий Азанов,,Photographer\r\rAnatoly Azanov was born on Jun...,Needs Vote
...,...,...,...,...,...
132,6900991,Vedic Social Empire,Ведическая социальная империя,Группа основана 21 декабря 2018 года Ампером А...,Needs Vote
133,6960465,Д.Н.О.,,Д.Н.О. (aka D.N.O. aka DNO aka ДНО) is an alte...,Needs Vote
134,7166614,TIR (4),True Industrial Romantic,"Группа TIR, в состав которой входят вокалист М...",Needs Vote
135,7231392,Trappist System Trio,,Прогрессив-рок трио TS3 образовано в мае 2017 ...,Needs Vote


In [82]:
drop_index('ft_profile', 'artist')

Время выполнения запроса: 0.1465 секунд


--------------------

# Транзакции